In [2]:
# ===== Colab: TF-IDF + Linear SVM (Calibrated) → Predict Aman.csv with confidences =====

import pandas as pd, numpy as np, requests, zipfile, io, re, os
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from sklearn.utils import shuffle
from google.colab import files

# -----------------------------
# 1) Load UCI SMS dataset
# -----------------------------
UCI_ZIP_URL = "http://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip"

def load_uci_sms():
    r = requests.get(UCI_ZIP_URL)
    with zipfile.ZipFile(io.BytesIO(r.content)) as zf:
        with zf.open("SMSSpamCollection") as f:
            df = pd.read_csv(f, sep="\t", header=None, names=["label", "text"])
    df["label"] = df["label"].map({"ham":0, "spam":1})
    df = shuffle(df, random_state=42).reset_index(drop=True)
    return df

uci = load_uci_sms()
X = uci["text"].astype(str).values
y = uci["label"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# -----------------------------
# 2) Build model: TF-IDF + LinearSVC (Calibrated → proba)
# -----------------------------
tfidf = TfidfVectorizer(
    lowercase=True,
    strip_accents="unicode",
    ngram_range=(1,2),
    min_df=2,
    max_df=0.95,
)

base_svm = LinearSVC(class_weight="balanced", random_state=42)
clf = CalibratedClassifierCV(estimator=base_svm, method="sigmoid", cv=5)

pipe = Pipeline([
    ("tfidf", tfidf),
    ("clf", clf),
])

print("Training TF-IDF + LinearSVC (calibrated)...")
pipe.fit(X_train, y_train)

# -----------------------------
# 3) Quick evaluation on UCI test split
# -----------------------------
y_prob = pipe.predict_proba(X_test)[:, 1]
y_pred = (y_prob >= 0.5).astype(int)

print("\n=== UCI Test Performance (baseline threshold=0.5) ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
try:
    print(f"ROC-AUC : {roc_auc_score(y_test, y_prob):.4f}")
except Exception:
    pass
print(classification_report(y_test, y_pred, target_names=["ham","spam"]))

# -----------------------------
# 4) Upload Aman.csv and predict
# -----------------------------
print("\n📤 Upload your Aman.csv (columns like date,time,sender,message typ/message type,message body,spam)")
uploaded = files.upload()
if not uploaded:
    raise RuntimeError("No file uploaded.")
csv_name = next(iter(uploaded))
print(f"Uploaded: {csv_name}")

def read_csv_robust(b: bytes) -> pd.DataFrame:
    for enc in ["utf-8", "utf-8-sig", "latin-1"]:
        try:
            return pd.read_csv(io.BytesIO(b), encoding=enc)
        except Exception:
            continue
    return pd.read_csv(io.BytesIO(b), engine="python")

df = read_csv_robust(uploaded[csv_name])

# Normalize column names
def norm(s: str) -> str:
    return re.sub(r"\s+", " ", s.strip().lower())

df.columns = [norm(c) for c in df.columns]
msg_candidates = ["message body","message","body","message_body","msg","text"]
msg_col = next((c for c in msg_candidates if c in df.columns), None)
if msg_col is None:
    raise ValueError(f"Couldn't find message text column. Looked for {msg_candidates}. Found {list(df.columns)}")

texts = df[msg_col].astype(str).fillna("")

# Predict probabilities
probs_spam = pipe.predict_proba(texts)[:, 1]

# Threshold (tune if needed)
THRESHOLD = 0.70  # try 0.6–0.75 depending on your data
pred_spam = (probs_spam >= THRESHOLD).astype(int)
pred_label = np.where(pred_spam==1, "spam", "ham")
confidence = np.maximum(probs_spam, 1.0 - probs_spam)  # model confidence in chosen class

# Build output
out = df.copy()
out["pred_spam"] = pred_spam           # 0=ham, 1=spam
out["pred_label"] = pred_label
out["spam_probability_%"] = (probs_spam * 100).round(2)
out["confidence_%"] = (confidence * 100).round(2)

# Diagnostics
print("\n--- Prediction summary on Aman.csv ---")
n = len(out)
spam_frac = float(out["pred_spam"].mean()) if n else 0.0
print(f"Predicted SPAM fraction @ {THRESHOLD:.2f}: {spam_frac*100:.2f}%  ({out['pred_spam'].sum()}/{n})")
if "sender" in out.columns:
    try:
        print("\nTop senders predicted as spam:")
        print(out.loc[out["pred_spam"]==1, "sender"].astype(str).str.strip().value_counts().head(15))
    except Exception:
        pass
if spam_frac > 0.85:
    print("\n⚠️ Still very high spam rate. Consider:")
    print("   - Raise THRESHOLD (e.g., 0.75).")
    print("   - Add a transactional override (banks/UPI/OTP → ham unless prob>=0.95).")
    print("   - Label ~200–400 of your messages and fine-tune on them.")

# Save + download
out_name = "aman_with_predictions_tfidf_svm.csv"
out.to_csv(out_name, index=False, encoding="utf-8")
print(f"\n✅ Saved: {out_name}")
files.download(out_name)


Training TF-IDF + LinearSVC (calibrated)...

=== UCI Test Performance (baseline threshold=0.5) ===
Accuracy: 0.9901
ROC-AUC : 0.9927
              precision    recall  f1-score   support

         ham       0.99      1.00      0.99       966
        spam       0.99      0.93      0.96       149

    accuracy                           0.99      1115
   macro avg       0.99      0.97      0.98      1115
weighted avg       0.99      0.99      0.99      1115


📤 Upload your Aman.csv (columns like date,time,sender,message typ/message type,message body,spam)


Saving Aman_all.csv to Aman_all.csv
Uploaded: Aman_all.csv

--- Prediction summary on Aman.csv ---
Predicted SPAM fraction @ 0.70: 27.20%  (556/2044)

Top senders predicted as spam:
sender
VM-SBIUPI    61
JK-SBIUPI    33
JD-SBIUPI    29
CP-SBIUPI    29
BA-SBIUPI    29
AD-AIRTEL    24
AX-ARTLTV    22
VK-SBIUPI    20
BP-SBIUPI    13
BW-SBIUPI    10
AD-650025     9
AD-ARWGOV     8
AD-AIRDOT     8
AD-650022     8
AD-SBIUPI     7
Name: count, dtype: int64

✅ Saved: aman_with_predictions_tfidf_svm.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>